# **Indicators4CLEWs_Clusters**  
**Original code:** Camilla Lo Giudice  
**Supervision:** Francesco Gardumi and Daniel Adshead  
**Funding:** IAM COMPACT


## Description of the notebook

The Notebook can be divided into six sections:  
- 1. Initialization
- 2. Creation of the directories
- 3. Conversion of model input *Data.txt* files into .csv files using otoole
- 4. Conversion of model *results.txt* files into 5 result files:
    - converted_data.csv
    - converted_data_TS.csv : contains all the results with different timeslices
    - converted_data_MoO.csv: contains all the results with different modes of operations
    - converted_data_Emissions.csv : contains all the results with emission types
    - converted_data_MoO_Emissions.csv : contains all the results with emission types and modes of operation
- 5. Indicators
- 6. Visualization

Each part will be further explained and the user will be guided through each step of the notebook. Users are required to customize the naming convention used in their specific model in the section *Configuration*.


## Configuration - to be filled by the user


In this part the user has to select the indicators of interest for the analysis. The input and output data used by the user might affect the outcome. For instance, if a result or imput data is missing it won't be possible to display the indicator. 

1. **How to select the indicator**  
For each indicator type the boolean *True* if the indicator is of interest, *False* otherwise

2. **Input the name of the variables**  
Each model can have similar but different naming conventions for the same technologies or commodities. Input the specific naming convention used in your model. For example *LNDFOR* if that is the name used for the land covered by forest. This will have to be done in the sub-section *Naming convention*. 

*Assumptions*

1) The workflow will assumes a standard format for the crops naming convention: **LND- Crop type - managment level - Irrigated/rainfed.**  
For example, if the model has an irrigated maize crop with a high managment level, the name will be: LND-MAI-H-I --> **LNDMAIHI**  

2) The same applies for other land types: all of them start with **LND- type.** For example forest will be LND - FOR --> **LNDFOR**

3) If there are crop imports, it is assumed that the naming convention will be **IMP** + the same crop names used for the other technolgies. For example for maize, it will be **IMPMAI**


### Indicators

Select *True* or *False* for each indicator

In [ ]:

Harvested_Area = True
BHI = True                                  #Biodiversity Habitat Index                                 
ws_aa = True                                #Average annual water stress



### Naming convention

Modify the following cell with the varibale names used in your model

In [ ]:
#Add list of Cluster numbers
"""
Here the user can add cluster numbers if the CLEWs model is not on a National scale. 
The assumed naming convention adds the cluster number after each technology - e.g LNDMAIHRC01 refers to the cluster C01 

Example list: Clusters = ["C01", "C02", "C03", "C04"]
"""
Clusters = ["C01", "C02", "C03", "C04"]

In [ ]:
# Forest cover indicator

Forest_Land = ["LNDFOR"]        #Technology for forest cover
Total_Land = ["LND"]         #Technology for the land resource



In [ ]:
#Harvested area indicator
"""
Create a list of crops consistent with the naming convention used in your model. 
This will ensure more flexibility to the workflow. 
The following list provides commonly used crop names in CLEWs models.

Maize = ["MAI"]
Rice = ["RIC"] 
Cereals = ["CER"]
Coffee = ["COF"]
Oilseeds = ["OIL"]
Pulses = ["PUL"]
Sugarcane = ["SUGC"]
Sorghum = ["SOR"]
Wheat = ["WHEAT"]
Barley = ["BAR"]
Soybeans = ["SOY"]
Other_Crops = ["OTC"]"

Example with this list: Crops = ["MAI", "RIC", "CER", "COF", "OIL", "PUL", "SUGC", "SOR", "WHEAT", "BAR", "SOY", "OTC"]
"""

Crops = ["CER", "WHE", "SOR", "COF", "BRL", "MAI", "OTC"]

Irrigated = ["I"]                   #Convention for irrigated crops rrigated crops
Rainfed = ["R"]                     #Convention for rainfed crops rrigated crops 


In [ ]:
Managment_level = True     # Select True if your model uses management levels and then define your naming convention

"""
Create a list of managment levels if Managment_level = True. 
The following list provides commonly used names in CLEWs models.

High_Management = ["H"]  
Intermediate_Management = ["I"]
Low_Management = ["L"]

Example list: Management_Levels = ["H", "I", "L"]
"""
Management_Levels = ["H", "L"]

In [ ]:
#Biodiversity Habitat Index (BHI)
# create a dictionary of original forest areas for each cluster consistent with the unit used in your model
Original_Forest_areas = {
    "C01": 165.31,
    "C02": 11.01,
    "C03": 275.68,
    "C04": 0.0
}

In [ ]:
#Average annual water stress

"""
Input the name of the commodities used for the water supply.
This entails water for public use, power sector or irrigation.
"""

#Initial_water_stress = 0.3  # Add the value of the initial water stress in the modelled country for the reference year.
                            # You can refer to the water risk atlas --> Water stress (https://www.wri.org/applications/aqueduct/water-risk-atlas)
                             
#Input the name of the model commodities  
Public_water = ["PUBWAT"]                  # Commodity name for public water
Industrial_water = ["INDWAT"]              # Commodity name for industrial water
#PWR_water = ["PWRWAT"]                     # Commodity name for cooling thermal power plants
Irrigation_water = ["AGRWAT"]              # Commodity name for irrigation water
Surface_water = ["WTRRUN"]                 # Commodity name for surface water
Ground_water = ["WTRGWT"]                  # Commodity name for groundwater
Evapotranspiration = ["WTREVT"]            # Commodity name for evapotranspiration

In [ ]:
#Initial water stress

"""
Fill this disctionary to calibrate the water stress in your model
"""

Initial_water_stress = {
    "C01": 0.2,
    "C02": 0.3,
    "C03": 0.1,
    "C04": 0.4
}



## 1. Initialization


In [ ]:
#Import Python modules or libraries

# Numerical
import numpy as np
import pandas as pd
import errno

# System & Other
import os
import re
from otoole import convert
import csv

#Plots
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

## 2. Creation of the directories  

In [ ]:
# Convert input data into csv files
ROOT_DIR = os.path.abspath(os.curdir)

#data and results input by the user
conversion_folder = "convert_from"
input = os.path.join(ROOT_DIR, conversion_folder + "\\"+ 'data.txt')
output = os.path.join(ROOT_DIR, conversion_folder + "\\"+ 'results.txt')
# Path to the configuration file
config_file = os.path.join(ROOT_DIR, "config_com.yaml")  # Update this path as needed

#folder where to save the converted csv files
Final_Data= os.path.join(ROOT_DIR, 'Data')
if not os.path.exists(Final_Data):
    try:
        os.makedirs(Final_Data)
    except OSError as exc: 
        if exc.errno != errno.EEXIST:
            raise

Final_results= os.path.join(ROOT_DIR, 'Results')
if not os.path.exists(Final_results):
    try:
        os.makedirs(Final_results)
    except OSError as exc: 
        if exc.errno != errno.EEXIST:
            raise

converted_input = os.path.join(ROOT_DIR, Final_Data + "\\"+ 'Model input')
if not os.path.exists(converted_input):
    try:
        os.makedirs(converted_input)
    except OSError as exc: 
        if exc.errno != errno.EEXIST:
            raise
converted_output = os.path.join(ROOT_DIR, Final_Data + "\\"+ 'Model output')
if not os.path.exists(converted_output):
    try:
        os.makedirs(converted_output)
    except OSError as exc: 
        if exc.errno != errno.EEXIST:
            raise

Plots = os.path.join(ROOT_DIR, 'Plots')
if not os.path.exists(Plots):
    try:
        os.makedirs(Plots)
    except OSError as exc: 
        if exc.errno != errno.EEXIST:
            raise


## 3. Conversion of model input Data.txt files into csv
This step uses the otoole environment for converting input data into csv files.Then the following three cells convert the results.txt file into three csv files: one with technologies with modes of operation, one with values per each time slice and one with only annual values (not considering neither modes of operation nor time-slices). The converted outputs will be stored into Data/Model input and in Data/Model output folders. The first one is used for the data converted from the data.txt file, while the second one contains the data converted from the results.txt file.

In [ ]:
#Fix txt file

def fix_txt_file(input):
    """
    Fixes the format of the data.txt file into a new data_fixed.txt. 
    The fixed parameters are `UDCTag`, `OperationalLifeStorage`, `StorageLevelStart`, and `UDCConstant`.
    
    """
    output_file = input.replace('.txt', '_fixed.txt')

    params_single_line_fix = [
        "UDCTag",
        "OperationalLifeStorage",
        "StorageLevelStart"
    ]

    params_udc_constant = ["UDCConstant"]

    with open(input, 'r') as f:
        lines = f.readlines()

    fixed_lines = []
    i = 0

    while i < len(lines):
        line = lines[i].strip()
        fixed = False

        for param in params_single_line_fix:
            if line.startswith(f"param {param} default"):
                if i + 3 < len(lines):
                    line2 = lines[i + 1].strip()
                    line4 = lines[i + 3].strip()
                    if line2 == ":=" and line4 == ";":
                        cleaned_line = re.sub(r'\s*:\s*$', '', line)
                        cleaned_line = re.sub(r'\s*:=\s*$', '', cleaned_line)
                        fixed_lines.append(f"{cleaned_line} :=\n")
                        fixed_lines.append(";\n")
                        i += 4
                        fixed = True
                        break

        for param in params_udc_constant:
            if line.startswith(f"param {param} default"):
                if i + 5 < len(lines):
                    line5 = lines[i + 5].strip()
                    if line5 == ";":
                        cleaned_line = re.sub(r'\s*:\s*$', '', line)
                        cleaned_line = re.sub(r'\s*:=\s*$', '', cleaned_line)
                        fixed_lines.append(f"{cleaned_line} :=\n")
                        fixed_lines.append(";\n")
                        i += 6
                        fixed = True
                        break

        if not fixed:
            fixed_lines.append(lines[i])
            i += 1

    with open(output_file, 'w') as f:
        f.writelines(fixed_lines)

    print(f"Fixed input file created: {output_file}")
    return output_file

fixed_input_file = fix_txt_file(input)
input_fixed = os.path.join(ROOT_DIR, conversion_folder + "\\"+ 'data_fixed.txt')
print(f"Fixed file created at: {fixed_input_file}")




In [ ]:
# Convert the input data to CSV files

def convert_input_to_csv(config_path, input_path, output_folder):
    """
    Converts a data.txt file into CSV files and stores them in the specified folder.

    Arguments
    ---------
    config_path : str
        Path to the configuration file.
    input_path : str
        Path to the input data_fixed.txt file.
    output_folder : str
        Path to the folder where the converted CSV files will be stored.

    Returns
    -------
    bool
        True if conversion was successful, False otherwise.
    """
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        try:
            os.makedirs(output_folder)
        except OSError as exc:
            if exc.errno != errno.EEXIST:
                raise

    # Perform the conversion
    try:
        success = convert(
            config=config_path,
            from_format="datafile",
            to_format="csv",
            from_path=input_path,
            to_path=output_folder,
            write_defaults=False,
            keep_whitespace=False,
        )
        return success
    except Exception as e:
        print(f"An error occurred during conversion: {e}")
        return False
    
# Convert the input data to CSV files
conversion_success = convert_input_to_csv(config_file, input_fixed, converted_input)

if conversion_success:
    print(f"Data conversion succeeded. CSV files are stored in: {converted_input}")
else:
    print("Conversion failed.")

## 4. Conversion of model results.txt file into csv


In [ ]:
#Convert the results to CSV files

def convert_results_to_csv(input_path, output_folder):
    """
    Converts the MUIO results.txt file into CSV files and stores them in the specified folder.

    Arguments
    ---------
    input_path : str
        Path to the input txt file.
    output_folder : str
        Path to the folder where the converted CSV files will be stored.

    Returns
    -------
    bool
        True if conversion was successful, False otherwise.
    """
  
    try:
        # Open the input txt file
        with open(input_path, 'r') as txt_file:
            lines = txt_file.readlines()

        # Prepare the output CSV file
        output_csv_path = os.path.join(output_folder, "converted_data.csv")
        with open(output_csv_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)

            # Write the header row
            csv_writer.writerow(["PARAMETER", "REGION", "TECHNOLOGY", "YEAR", "VALUE",])

            # Parse each line in the txt file
            for line in lines:
                # Split the line into columns based on fixed-width formatting
                parts = line.strip().split()
                if len(parts) < 3:
                    continue  # Skip invalid lines

                # Extract the parameter, and values
                parameter_info = parts[1]
                value = parts[2]

                # Parse the parameter info (e.g., NewCapacity(RE1,PWRHYDLRG,2020))
                if "(" in parameter_info and ")" in parameter_info:
                    parameter, details = parameter_info.split("(")
                    details = details.strip(")").split(",")
                    if len(details) == 3:
                        region, technology, year = details
                        # Write the parsed data to the CSV file
                        csv_writer.writerow([parameter, region, technology, year, value,])

        return True
    except Exception as e:
        print(f"An error occurred during conversion: {e}")
        return False

# Call the function to convert the custom txt file to CSV
conversion_success = convert_results_to_csv(output, converted_output)

if conversion_success:
    print(f"Data conversion succeeded. CSV files are stored in: {converted_output}")
else:
    print("Conversion failed.")

In [ ]:
#Convert the results to CSV files with TimeSlices

def convert_resultsTS_to_csv(input_path, output_folder):
    """
    Converts the MUIO results.txt file into CSV files with time-slices and stores them in the specified folder.

    Arguments
    ---------
    input_path : str
        Path to the input txt file.
    output_folder : str
        Path to the folder where the converted CSV files will be stored.

    Returns
    -------
    bool
        True if conversion was successful, False otherwise.
    """
  
    try:
        # Open the input txt file
        with open(input_path, 'r') as txt_file:
            lines = txt_file.readlines()

        # Prepare the output CSV file
        output_csv_TS_path = os.path.join(output_folder, "converted_data_TS.csv")
        with open(output_csv_TS_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)

            # Write the header row
            csv_writer.writerow(["PARAMETER", "REGION", "TECHNOLOGY","TIMESLICE","YEAR", "VALUE",])

            # Parse each line in the txt file
            for line in lines:
                # Split the line into columns based on fixed-width formatting
                parts = line.strip().split()
                if len(parts) < 4:
                    continue  # Skip invalid lines

                # Extract the parameter, and values
                parameter_info = parts[1]
                value = parts[2]

                # Parse the parameter info 
                if "(" in parameter_info and ")" in parameter_info:
                    parameter, details = parameter_info.split("(")
                    details = details.strip(")").split(",")
                    if len(details) == 4:
                        region, technology, time_slice, year = details
                        
                        # Skip if the time-slice is a number
                        if time_slice.isdigit():
                            continue

                        # Skip if the parameter is "AnnualTechnologyEmission" or "InputToTotalCapacity"
                        if parameter in ["AnnualTechnologyEmission", "InputToTotalCapacity", "Demand"]:
                            continue

                        # Write the parsed data to the CSV file
                        csv_writer.writerow([parameter, region, technology, time_slice, year, value,])

        return True
    except Exception as e:
        print(f"An error occurred during conversion: {e}")
        return False

# Call the function to convert the custom txt file to CSV
conversion_success = convert_resultsTS_to_csv(output, converted_output)

if conversion_success:
    print(f"Data conversion succeeded. CSV files are stored in: {converted_output}")
else:
    print("Conversion failed.")

In [ ]:
#Convert the results to CSV files with Modes of operation

def convert_resultsMoO_to_csv(input_path, output_folder):
    """
    Converts the MUIO results.txt with modes of operation (MoO) into CSV files with time-slices and stores them in the specified folder.

    Arguments
    ---------
    input_path : str
        Path to the input txt file.
    output_folder : str
        Path to the folder where the converted CSV files will be stored.

    Returns
    -------
    bool
        True if conversion was successful, False otherwise.
    """
  
    try:
        # Open the input txt file
        with open(input_path, 'r') as txt_file:
            lines = txt_file.readlines()

        # Prepare the output CSV file
        output_csv_MoO_path = os.path.join(output_folder, "converted_data_MoO.csv")
        with open(output_csv_MoO_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)

            # Write the header row
            csv_writer.writerow(["PARAMETER", "REGION", "TECHNOLOGY","MODE_OF_OPERATION","YEAR", "VALUE",])

            # Parse each line in the txt file
            for line in lines:
                # Split the line into columns based on fixed-width formatting
                parts = line.strip().split()
                if len(parts) < 4:
                    continue  # Skip invalid lines

                # Extract the parameter, and values
                parameter_info = parts[1]
                value = parts[2]

                # Parse the parameter info 
                if "(" in parameter_info and ")" in parameter_info:
                    parameter, details = parameter_info.split("(")
                    details = details.strip(")").split(",")
                    if len(details) == 4:
                        region, technology, mode_of_operation, year = details
                        if mode_of_operation.isdigit(): # Modes of operaion can be only > 1
                            # Write the parsed data to the CSV file
                            csv_writer.writerow([parameter, region, technology, mode_of_operation, year, value,])

        return True
    except Exception as e:
        print(f"An error occurred during conversion: {e}")
        return False

# Call the function to convert the custom txt file to CSV
conversion_success = convert_resultsMoO_to_csv(output, converted_output)

if conversion_success:
    print(f"Data conversion succeeded. CSV files are stored in: {converted_output}")
else:
    print("Conversion failed.")

In [ ]:
#Convert the emission results to CSV files 

def convert_resultsEMI_to_csv(input_path, output_folder):
    """
    Converts the MUIO results.txt file into CSV files with emissions and stores them in the specified folder.

    Arguments
    ---------
    input_path : str
        Path to the input txt file.
    output_folder : str
        Path to the folder where the converted CSV files will be stored.

    Returns
    -------
    bool
        True if conversion was successful, False otherwise.
    """
  
    try:
        # Open the input txt file
        with open(input_path, 'r') as txt_file:
            lines = txt_file.readlines()

        # Prepare the output CSV file
        output_csv_EMI_path = os.path.join(output_folder, "converted_data_Emissions.csv")
        with open(output_csv_EMI_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)

            # Write the header row
            csv_writer.writerow(["PARAMETER", "REGION", "TECHNOLOGY","EMISSION","YEAR", "VALUE",])

            # Parse each line in the txt file
            for line in lines:
                # Split the line into columns based on fixed-width formatting
                parts = line.strip().split()
                if len(parts) < 4:
                    continue  # Skip invalid lines

                # Extract the parameter, and values
                parameter_info = parts[1]
                value = parts[2]

                # Parse the parameter info 
                if "(" in parameter_info and ")" in parameter_info:
                    parameter, details = parameter_info.split("(")
                    details = details.strip(")").split(",")
                    if len(details) == 4:
                        region, technology, emission, year = details
                        
                        # Skip if the time-slice is a number
                        if emission.isdigit():
                            continue

                        # Skip if the parameter is "AnnualTechnologyEmission" or "InputToTotalCapacity"
                        if parameter in [ "RateOfTotalActivity","InputToTotalCapacity", "Demand"]:
                            continue

                        # Write the parsed data to the CSV file
                        csv_writer.writerow([parameter, region, technology, emission, year, value,])

        return True
    except Exception as e:
        print(f"An error occurred during conversion: {e}")
        return False

# Call the function to convert the custom txt file to CSV
conversion_success = convert_resultsEMI_to_csv(output, converted_output)

if conversion_success:
    print(f"Data conversion succeeded. CSV files are stored in: {converted_output}")
else:
    print("Conversion failed.")

In [ ]:
#Convert the emission results to CSV files with Modes of Operation (MoO)

def convert_resultsEMIMoO_to_csv(input_path, output_folder):
    """
    Converts the MUIO results.txt file into CSV files with emissions with MoO and stores them in the specified folder.

    Arguments
    ---------
    input_path : str
        Path to the input txt file.
    output_folder : str
        Path to the folder where the converted CSV files will be stored.

    Returns
    -------
    bool
        True if conversion was successful, False otherwise.
    """
    try:
        # Open the input txt file
        with open(input_path, 'r') as txt_file:
            lines = txt_file.readlines()

        # Prepare the output CSV file
        output_csv_EMIMoO_path = os.path.join(output_folder, "converted_data_MoO_Emissions.csv")
        with open(output_csv_EMIMoO_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)

            # Write the header row
            csv_writer.writerow(["PARAMETER", "REGION", "TECHNOLOGY", "EMISSION", "MODE_OF_OPERATION", "YEAR", "VALUE"])

            # Parse each line in the txt file
            for line in lines:
                # Split the line into columns based on fixed-width formatting
                parts = line.strip().split()
                if len(parts) < 3:  # Ensure the line has enough parts
                    #print(f"Skipping line (not enough parts): {line.strip()}")
                    continue

                # Extract the parameter and value
                parameter_info = parts[1]
                value = parts[2]

                # Parse the parameter info
                if "(" in parameter_info and ")" in parameter_info:
                    parameter, details = parameter_info.split("(")
                    details = details.strip(")").split(",")
                    if len(details) == 5:  # Ensure the details have the expected structure
                        region, technology, emission, mode, year = details

                        if "Emission" not in parameter:
                            continue

                        if not mode.strip().isdigit():
                            continue

                        if parameter in [ "RateOfActivity"]:
                            continue

                        # Write the parsed data to the CSV file
                        csv_writer.writerow([parameter, region, technology, emission, mode, year, value])
                    

        return True
    except Exception as e:
        print(f"An error occurred during conversion: {e}")
        return False

# Call the function to convert the custom txt file to CSV
conversion_success = convert_resultsEMIMoO_to_csv(output, converted_output)

if conversion_success:
    print(f"Data conversion succeeded. CSV files are stored in: {converted_output}")
else:
    print("Conversion failed.")

## 5. Indicators


### 5.1 Harvested area with clusters  
This indicator shows the share of total harvested land (both rainfed or irrigated), calculated as <br><br>
$$Harvested\_Area = \frac{Total\_land\_harvested}{Total\_land}$$

Note that the type of crops might vary greatly depending on the case study country. If a specific crop is missing from the ones proposed, add it in the configuration cell. Additionally, models can vary in complexity. A part from distinguishing between rainfed and irrigated, it is possible to model also three generic input/management levels defined in the GAEZ v4.0 model documentation:  
- *Low level input:*  
   Traditional managment assumption  
- *Intermediate level inputs:*  
   Improved managment assumption
- *High level input:*  
   Advanced managment assumption

For this indicator the configuration cell will be split in three main cells: managment level, crop types and irrigated/rainfed. This is due to the high flexiility that each user has in shaping the land use. The workflow will assume the standard format used for naming crops: **LND- Crop type - managment level - Irrigated/rainfed.**  

For example, if the model has an irrigated maize crop with a high managment level, the name will be: LND-MAI-H-I --> **LNDMAIHI**  


For the GAEZ v4 model documentation refer to https://openknowledge.fao.org/server/api/core/bitstreams/6b7b9b4a-dbac-4af4-a2cb-26aff33a30e5/content 


##### Harvested area indicator


In [ ]:
# Run the cell for the harvested Area indicator
if Harvested_Area == False:
    exit()

# Paths
input_csv = os.path.join(converted_output, 'converted_data.csv')
output_csv = os.path.join(Final_results, 'Harvested_area.csv')

# Read the input CSV file
try:
    data = pd.read_csv(input_csv)
except FileNotFoundError:
    print(f"Error: The file {input_csv} does not exist.")
    exit()

# Filter the data for the parameter "TotalTechnologyAnnualActivity"
filtered_data = data[data["PARAMETER"] == "TotalTechnologyAnnualActivity"]

# Initialize a dictionary to store the results
results = {"INDICATOR": [],"CLUSTER": [], "VALUE": [], "UNIT": [], "YEAR": []}


for cluster in Clusters:
    # derive numeric cluster code used by crop technologies (e.g. "01" from "C01")
    crop_cluster = cluster[1:] if cluster.startswith("C") else cluster

    # Build the regular expression parts
    crop_regex = "|".join(Crops)
    irrigation_regex = "|".join(Irrigated + Rainfed)

    # Check if management levels are used and build technology regex for the current cluster's crop code
    if Managment_level:
        management_regex = "|".join(Management_Levels)
        technology_regex = rf"LND({crop_regex})({management_regex})({irrigation_regex})({crop_cluster})"
    else:
        technology_regex = rf"LND({crop_regex})({irrigation_regex})({crop_cluster})"

    # calculate the ratio
    for year in filtered_data["YEAR"].unique():
        # Filter data for the current year
        year_data = filtered_data[filtered_data["YEAR"] == year]

        # Filter technologies that match the naming convention for THIS cluster (using numeric cluster part for crops)
        harvested_area_data = year_data[year_data["TECHNOLOGY"].str.contains(technology_regex, flags=re.IGNORECASE, na=False)]

        # Get the total land value for THIS cluster. Total_Land entries are expected to contain cluster like "C01".
        total_land_candidates = [t for t in Total_Land if cluster in t]
        if total_land_candidates:
            total_land_data = year_data[year_data["TECHNOLOGY"].isin(total_land_candidates)]
        else:
            # fallback: try matching by numeric cluster code if Total_Land doesn't contain full cluster string
            total_land_data = year_data[year_data["TECHNOLOGY"].str.contains(crop_cluster, na=False)]

        total_land_value = total_land_data["VALUE"].sum()
        harvested_area_sum = harvested_area_data["VALUE"].sum()

        # Calculate the ratio if total land value is not zero
        if total_land_value != 0:
            ratio = round((harvested_area_sum / total_land_value) * 100, 3)
            results["INDICATOR"].append("HarvestedArea")
            results["CLUSTER"].append(cluster)
            results["VALUE"].append(ratio)
            results["UNIT"].append("%")
            results["YEAR"].append(year)
# Convert the results dictionary to a DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(['CLUSTER', 'YEAR']).reset_index(drop=True)

# Save the results to a CSV file
results_df.to_csv(output_csv, index=False)
print(f"Harvested area indicators have been successfully saved to {output_csv}.")

### 5.2 Biodiversity with clusters

The Biodiversity Habitat Index (BHI) assesses the effects of habitat loss, degradation, and fragmentation on the retention of terrestrial biodiversity in a region. The indicator exploits the species-area relationship: <br><br><br>


 $$BHI = \frac{Forest\_Area\_Retained}{Original\_Forest\_Area}^{0.25} $$
 

 

 In this analysis, 0.25 represents the exponent *z* of the species–area relationship, a widely accepted value for this type of assessment.
 The results show the % increase (or decrease) in biodiversity.
 Being CLEWs models not spatially explicit, we refered to the simplified relationship presented by Simon Ferrier et al. (2004). For more information refer to: BioScience, Volume 54, Issue 12, December 2004, Pages 1101–1109, https://doi.org/10.1641/0006-3568(2004)054[1101:MMOTBF]2.0.CO;2 




##### BHI indicator


In [ ]:
# Exit if the indicator is not of interest
if BHI == False:
    exit()

# Paths
input_csv = os.path.join(converted_output, 'converted_data.csv')
BHI_output_csv = os.path.join(Final_results, 'BHI.csv')

# Read the input CSV file
try:
    data = pd.read_csv(input_csv)
except FileNotFoundError:
    print(f"Error: The file {input_csv} does not exist.")
    exit()

# Filter the data for the parameter "TotalTechnologyAnnualActivity"
filtered_data = data[data["PARAMETER"] == "TotalTechnologyAnnualActivity"]

# Initialize a dictionary to store the results
results = {"INDICATOR": [],"CLUSTER": [], "VALUE": [], "UNIT": [], "YEAR": []}

for cluster in Clusters:
    forest_tech = [t + cluster for t in Forest_Land]
    # Loop through each year and calculate the BHI
    for year in filtered_data["YEAR"].unique():
        # Filter data for the current year
        year_data = filtered_data[filtered_data["YEAR"] == year]
        
        # Get the value for forest land
        lndfor_value = year_data[year_data["TECHNOLOGY"].isin(forest_tech)]["VALUE"].sum()
        
        # Calculate the BHI
        original_forest_value = Original_Forest_areas[cluster]
        if original_forest_value is None:
            print(f"Warning: Original forest area for cluster {cluster} is not defined. Skipping BHI calculation for this cluster.")
            continue
        if original_forest_value != 0:
            ratio = round(((lndfor_value / original_forest_value) ** 0.25)*100, 3)
            results["INDICATOR"].append("BHI")
            results["CLUSTER"].append(cluster)
            results["VALUE"].append(round(ratio, 3))
            results["UNIT"].append("%")
            results["YEAR"].append(year)

# Convert the results dictionary to a DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(['CLUSTER', 'YEAR']).reset_index(drop=True)
# Save the results to a CSV file
results_df.to_csv(BHI_output_csv, index=False)

print(f"BHI indicator has been successfully saved to {BHI_output_csv}.")

### 5.3 Average annual water stress and relative water demand

The water stress can be calculated as the ratio of the total water demand over the total available renewable surface and groundwater supplies (also known as relative water demand). It is suitable for models with low temporal resolution. The steps for calculating the indicator are the following: <br><br>
*Step 1: calculate the relative water demand* <br><br><br>

$$r = \frac{Total\_Annual\_Gross\_Demand}{Total\_Annual\_Available\_Water} $$
 

The annual demand includes public and industrial uses and the demand associated to the power and agricultural sectors.

*Step 2: calculate the water average annual stress*<br><br><br>

$$ws_{aa} = \min(1, \max(0, r))$$
 
*Step 3: conversion to risk categories*<br><br><br>

$$Average\_Annual\_score = \max\left(0, \min\left(5, \frac{ln(ws_{aa})-ln(0.1)}{ln(2)} +1\right)\right)$$



The risk cathegory follows the same values and methodlogy used by WRI and then adapted to a CLEWs model. The code returns two csv files: one for the relative water demand and one with the average water stress per each modelled year.


For more information about the methodology adopted by WRI refer to https://files.wri.org/d8/s3fs-public/2023-08/aqueduct-40-technical-note.pdf?VersionId=G_TxTR2LAnlgXGzy7xtdUP_5lmkXJY7d


In [ ]:
# Exit if the indicator is not of interest
if ws_aa == False:
    exit()

# Paths
IAR_csv = os.path.join(converted_input, "InputActivityRatio.csv")
OAR_csv = os.path.join(converted_input, "OutputActivityRatio.csv")
Water_annual_demand_sp_csv = os.path.join(converted_input, "SpecifiedAnnualDemand.csv")
Water_annual_demand_annual_csv = os.path.join(converted_input, "AccumulatedAnnualDemand.csv")
output_Activity_csv = os.path.join(converted_output, "converted_data.csv")
output_Activity_MoO_csv = os.path.join(converted_output, "converted_data_MoO.csv")
Year_split= os.path.join(converted_input, "YearSplit.csv")
r_csv = os.path.join(Final_results, "Relative_annual_water_demand.csv")
Annual_average_ws_score = os.path.join(Final_results, "Annual_average_ws_score.csv")

# Read the input CSV files
try:
    data_IAR = pd.read_csv(IAR_csv)
except FileNotFoundError:
    print(f"Error: The file {IAR_csv} does not exist.")
    exit()
if data_IAR.empty:
    print("The {IAR_csv} file is empty.")
    exit()

try:
    data_OAR = pd.read_csv(OAR_csv)
except FileNotFoundError:
    print(f"Error: The file {OAR_csv} does not exist.")
    exit()
if data_OAR.empty:
    print("The {OAR_csv} file is empty.")
    exit()

try:
    data_Water_demand_sp = pd.read_csv(Water_annual_demand_sp_csv)
except FileNotFoundError:
    print(f"Error: The file {Water_annual_demand_sp_csv} does not exist.")
    exit()
if data_Water_demand_sp[ data_Water_demand_sp["COMMODITY"].isin(Public_water) ].empty:
    print(f"The commodity {Public_water} is not present in {Water_annual_demand_sp_csv}.")

try:
    data_Water_demand_annual = pd.read_csv(Water_annual_demand_annual_csv)
except FileNotFoundError:
    print(f"Error: The file {Water_annual_demand_annual_csv} does not exist.")
    exit()
if data_Water_demand_annual.empty:
    print("The {Water_annual_demand_annual_csv} file is empty.")
    exit()

try:
    data_activity = pd.read_csv(output_Activity_csv)
except FileNotFoundError:
    print(f"Error: The file {output_Activity_csv} does not exist.")
    exit()
if data_activity.empty:
    print("The {output_Activity_csv} file is empty.")
    exit()


try:
    data_activity_MoO = pd.read_csv(output_Activity_MoO_csv)
except FileNotFoundError:
    print(f"Error: The file {output_Activity_MoO_csv} does not exist.")
    exit()
if data_activity_MoO.empty:
    print("The {output_Activity_MoO_csv} file is empty.")
    exit()

##### Water demand 

In [ ]:
# 1. Water demand for industrial demand
# Exit if the indicator is not of interest
if ws_aa == False:
    exit()

# Load Year_split data
data_Year_split = pd.read_csv(Year_split)

# Initialize empty list to store results
results = []

for cluster in Clusters:
    # derive numeric cluster code used by crop technologies (e.g. "01" from "C01")
    demand_cluster = cluster[1:] if cluster.startswith("C") else cluster
    
    # Create industrial water commodity name for this cluster
    industrial_demand = [t + demand_cluster for t in Industrial_water]
    
    # Get annual demand data
    filtered_demand = data_Water_demand_annual[data_Water_demand_annual["COMMODITY"].isin(industrial_demand)]
    
    # Process each year
    unique_years = sorted(data_Year_split["YEAR"].unique())
    for year in unique_years:
        year_demand = filtered_demand[filtered_demand["YEAR"] == year]
        if not year_demand.empty:
            annual_value = year_demand["VALUE"].values[0]
            # Get year split values for this year
            year_splits = data_Year_split[data_Year_split["YEAR"] == year]
            
            # Calculate for each season
            for season in [1, 2]:
                # Get timeslices for this season (S11, S12, S13 for season 1, etc.)
                season_splits = year_splits[year_splits["TIMESLICE"].str.startswith(f'S{season}')]
                
                # Calculate seasonal demand (sum of timeslice values * annual demand)
                seasonal_demand = annual_value * season_splits["VALUE"].sum()
                
                # Append result
                for commodity in industrial_demand:
                    results.append({
                        "COMMODITY": commodity,
                        "CLUSTER": cluster,
                        "SEASON": f"{season}",
                        "VALUE": seasonal_demand,
                        "YEAR": year
                    })

# Convert results to DataFrame
result_industrial_demand = pd.DataFrame(results)
# Display the results
display(result_industrial_demand)

In [ ]:
#Irrigation demand

#Input the name of the model commodities  
Public_water = ["PUBWAT"]                  # Commodity name for public water
Industrial_water = ["INDWAT"]              # Commodity name for industrial water
#PWR_water = ["PWRWAT"]                     # Commodity name for cooling thermal power plants
Irrigation_water = ["AGRWAT"]              # Commodity name for irrigation water
Surface_water = ["WTRRUN"]                 # Commodity name for surface water
Ground_water = ["WTRGWT"]                  # Commodity name for groundwater
Evapotranspiration = ["WTREVT"]            # Commodity name for evapotranspiration


# Initialize empty list to store results
irrigation_results = []

# Get unique years from the data
unique_years = sorted(data_activity_MoO["YEAR"].unique())

# Process each technology activity
for _, activity_row in data_activity_MoO.iterrows():
    technology = activity_row["TECHNOLOGY"]
    year = activity_row["YEAR"]
    mode = activity_row["MODE_OF_OPERATION"]
    activity_value = activity_row["VALUE"]
    
    # Extract cluster number from technology (last 2 digits)
    cluster = technology[-2:] if technology[-2:].isdigit() else None
    if cluster is None:
        continue
        
    # Create corresponding irrigation water commodity name
    #irrigation_water = f"AGRWAT{cluster}"
    
    irrigation_water = str(Irrigation_water[0]) + cluster
    
    # Get corresponding IAR value for the same technology and year
    iar_value = data_IAR[
        (data_IAR["TECHNOLOGY"] == technology) & 
        (data_IAR["YEAR"] == year) &
        (data_IAR["MODE_OF_OPERATION"] == mode) &
        (data_IAR["COMMODITY"] == irrigation_water)
    ]["VALUE"]
    
    if not iar_value.empty:
        # Calculate total water demand
        water_demand = activity_value * iar_value.iloc[0]
        
        # Determine season based on MODE_OF_OPERATION
        season = f"{mode}"  # assuming mode 1 -> season 1, mode 2 -> season 2
        
        # Add to results
        irrigation_results.append({
            "TECHNOLOGY": technology,
            "CLUSTER": f"C{cluster}",  # Adding 'C' prefix to match Clusters format
            "SEASON": season,
            "VALUE": water_demand,
            "YEAR": year
        })

# Convert to DataFrame and remove duplicates
irrigation_result = pd.DataFrame(irrigation_results).drop_duplicates()

# Reset index to remove the large index numbers
irrigation_result = irrigation_result.reset_index(drop=True)

# Aggregate by CLUSTER, SEASON, YEAR
result_irrigation_demand = (
    irrigation_result
    .groupby(['CLUSTER', 'SEASON', 'YEAR'], as_index=False)['VALUE']
    .sum()
)

result_irrigation_demand = result_irrigation_demand[['CLUSTER', 'SEASON', 'VALUE', 'YEAR']]

# Sort 
result_irrigation_demand = result_irrigation_demand.sort_values(['CLUSTER', 'YEAR', 'SEASON']).reset_index(drop=True)

# Display the summary
display(result_irrigation_demand)

In [ ]:
# 4. Water for public use

# Exit if the indicator is not of interest
if ws_aa == False:
    exit()

# Load Year_split data
data_Year_split = pd.read_csv(Year_split)

# Initialize empty list to store results
results = []

for cluster in Clusters:
    # derive numeric cluster code used by crop technologies (e.g. "01" from "C01")
    demand_cluster = cluster[1:] if cluster.startswith("C") else cluster
    
    # Create industrial water commodity name for this cluster
    public_demand = [t + demand_cluster for t in Public_water]
    
    # Get annual demand data
    filtered_demand = data_Water_demand_annual[data_Water_demand_annual["COMMODITY"].isin(public_demand)]
    
    # Process each year
    unique_years = sorted(data_Year_split["YEAR"].unique())
    for year in unique_years:
        year_demand = filtered_demand[filtered_demand["YEAR"] == year]
        if not year_demand.empty:
            annual_value = year_demand["VALUE"].values[0]
            # Get year split values for this year
            year_splits = data_Year_split[data_Year_split["YEAR"] == year]
            
            # Calculate for each season
            for season in [1, 2]:
                # Get timeslices for this season (S11, S12, S13 for season 1, etc.)
                season_splits = year_splits[year_splits["TIMESLICE"].str.startswith(f'S{season}')]
                
                # Calculate seasonal demand (sum of timeslice values * annual demand)
                seasonal_demand = annual_value * season_splits["VALUE"].sum()
                
                # Append result
                for commodity in public_demand:
                    results.append({
                        "COMMODITY": commodity,
                        "CLUSTER": cluster,
                        "SEASON": f"{season}",
                        "VALUE": seasonal_demand,
                        "YEAR": year
                    })

# Convert results to DataFrame
result_public_demand = pd.DataFrame(results)

# Display the results
display(result_public_demand)


In [ ]:
#5. Final water demand

# Exit if the indicator is not of interest
if ws_aa == False:
    exit()

# Ensure all merge columns are of the same type (string for CLUSTER, int for YEAR, int for SEASON)
def _normalize_types(df):
    df = df.copy()
    df['CLUSTER'] = df['CLUSTER'].astype(str)
    df['YEAR'] = df['YEAR'].astype(int)
    # Convert SEASON to int if possible, else to string
    if df['SEASON'].dtype != 'int64' and df['SEASON'].dtype != 'Int64':
        df['SEASON'] = pd.to_numeric(df['SEASON'], errors='coerce').astype('Int64')
    return df

dfs = [result_irrigation_demand, result_industrial_demand, result_public_demand]
dfs = [_normalize_types(df) for df in dfs]

# Rename VALUE columns to distinguish them
dfs[0] = dfs[0].rename(columns={'VALUE': 'VALUE_IRRIGATION'})
dfs[1] = dfs[1].rename(columns={'VALUE': 'VALUE_INDUSTRIAL'})
dfs[2] = dfs[2].rename(columns={'VALUE': 'VALUE_PUBLIC'})

# Merge all three DataFrames on CLUSTER, SEASON, YEAR (outer join)
merged = pd.merge(dfs[0], dfs[1], on=['CLUSTER', 'SEASON', 'YEAR'], how='outer')
merged = pd.merge(merged, dfs[2], on=['CLUSTER', 'SEASON', 'YEAR'], how='outer')

# Fill NaN with 0 for summing
for col in ['VALUE_IRRIGATION', 'VALUE_INDUSTRIAL', 'VALUE_PUBLIC']:
    if col in merged.columns:
        merged[col] = merged[col].fillna(0)

# Calculate total VALUE
merged['VALUE'] = merged['VALUE_IRRIGATION'] + merged['VALUE_INDUSTRIAL'] + merged['VALUE_PUBLIC']

# Select and reorder columns
final_water_demand = merged[['CLUSTER', 'SEASON', 'YEAR', 'VALUE']].sort_values(['CLUSTER', 'YEAR', 'SEASON']).reset_index(drop=True)

display(final_water_demand)

##### Available water

The methodology used by WRI considers the total available renewable surface and groundwater supplies estimated through a hydrological model. In a CLEWs model this supply comes from the output activity ratios associated to all the land uses.

In [ ]:
# 1. Surface water

# Initialize empty list to store results
Surface_water_results = []
SurfaceWater = os.path.join(Final_results, 'Surface_water.csv')

# Get unique years from the data
unique_years = sorted(data_activity_MoO["YEAR"].unique())

# List of technology prefixes that need season duplication when mode == 1
prefixes = ["LNDOTHC", "LNDBLTC", "LNDFORC", "LNDWATC", "LNDGRSC"]

# Process each technology activity
for _, activity_row in data_activity_MoO.iterrows():
    technology = activity_row["TECHNOLOGY"]
    year = activity_row["YEAR"]
    mode = activity_row["MODE_OF_OPERATION"]
    activity_value = activity_row["VALUE"]
    
    # Extract cluster number from technology (last 2 digits)
    cluster = technology[-2:] if technology[-2:].isdigit() else None
    if cluster is None:
        continue
        
    surface_water = str(Surface_water[0]) + cluster

    # Get corresponding OAR value for the same technology and year
    oar_value = data_OAR[
        (data_OAR["TECHNOLOGY"] == technology) & 
        (data_OAR["YEAR"] == year) &
        (data_OAR["MODE_OF_OPERATION"] == mode) &
        (data_OAR["COMMODITY"] == surface_water)
    ]["VALUE"]
    
    if not oar_value.empty:
        # Calculate total surface water supply
        sur_wat_supply = activity_value * oar_value.iloc[0]
        
        # Check if this is a special technology that needs season duplication
        is_special_tech = any(technology.startswith(p) for p in prefixes)
        
        # For special technologies with mode == 1, duplicate the value for both seasons
        if is_special_tech and mode == 1:
            Surface_water_results.append({
                "TECHNOLOGY": technology,
                "CLUSTER": f"C{cluster}",
                "SEASON": 1,
                "VALUE": sur_wat_supply,
                "YEAR": year
            })
            Surface_water_results.append({
                "TECHNOLOGY": technology,
                "CLUSTER": f"C{cluster}",
                "SEASON": 2,
                "VALUE": sur_wat_supply,
                "YEAR": year
            })
        else:
            # For all other cases (non-special tech or mode != 1), handle normally
            try:
                season = int(mode)
            except Exception:
                season = mode
                
            Surface_water_results.append({
                "TECHNOLOGY": technology,
                "CLUSTER": f"C{cluster}",
                "SEASON": season,
                "VALUE": sur_wat_supply,
                "YEAR": year
            })

# Convert to DataFrame and remove duplicates
Surface_water_result = pd.DataFrame(Surface_water_results).drop_duplicates()

# Reset index to remove the large index numbers
Surface_water_result = Surface_water_result.reset_index(drop=True)

# Aggregate by CLUSTER, SEASON, YEAR
result_Surface_water_demand = (
    Surface_water_result
    .groupby(['CLUSTER', 'SEASON', 'YEAR'], as_index=False)['VALUE']
    .sum()
)

result_Surface_water_demand = result_Surface_water_demand[['CLUSTER', 'SEASON', 'VALUE', 'YEAR']]

# Sort 
result_Surface_water_demand = result_Surface_water_demand.sort_values(['CLUSTER', 'YEAR', 'SEASON']).reset_index(drop=True)

# Display the summary
display(result_Surface_water_demand)



result_Surface_water_demand.to_csv(SurfaceWater, index=False)



In [ ]:
# 2. Groundwater
# Initialize empty list to store results
Groundwater_results = []

# Get unique years from the data
unique_years = sorted(data_activity_MoO["YEAR"].unique())

# List of technology prefixes that need season duplication when mode == 1
prefixes = ["LNDOTHC", "LNDBLTC", "LNDFORC", "LNDWATC", "LNDGRSC"]

# Process each technology activity
for _, activity_row in data_activity_MoO.iterrows():
    technology = activity_row["TECHNOLOGY"]
    year = activity_row["YEAR"]
    mode = activity_row["MODE_OF_OPERATION"]
    activity_value = activity_row["VALUE"]
    
    # Extract cluster number from technology (last 2 digits)
    cluster = technology[-2:] if technology[-2:].isdigit() else None
    if cluster is None:
        continue
        
    ground_water = str(Ground_water[0]) + cluster

    # Get corresponding OAR value for the same technology and year
    oar_value = data_OAR[
        (data_OAR["TECHNOLOGY"] == technology) & 
        (data_OAR["YEAR"] == year) &
        (data_OAR["MODE_OF_OPERATION"] == mode) &
        (data_OAR["COMMODITY"] == ground_water)
    ]["VALUE"]
    
    if not oar_value.empty:
        # Calculate total groundwater supply
        g_wat_supply = activity_value * oar_value.iloc[0]
        
        # Check if this is a special technology that needs season duplication
        is_special_tech = any(technology.startswith(p) for p in prefixes)
        
        # For special technologies with mode == 1, duplicate the value for both seasons
        if is_special_tech and mode == 1:
            Groundwater_results.append({
                "TECHNOLOGY": technology,
                "CLUSTER": f"C{cluster}",
                "SEASON": 1,
                "VALUE": g_wat_supply,
                "YEAR": year
            })
            Groundwater_results.append({
                "TECHNOLOGY": technology,
                "CLUSTER": f"C{cluster}",
                "SEASON": 2,
                "VALUE": g_wat_supply,
                "YEAR": year
            })
        else:
            # For all other cases (non-special tech or mode != 1), handle normally
            try:
                season = int(mode)
            except Exception:
                season = mode
                
            Groundwater_results.append({
                "TECHNOLOGY": technology,
                "CLUSTER": f"C{cluster}",
                "SEASON": season,
                "VALUE": g_wat_supply,
                "YEAR": year
            })

# Convert to DataFrame and remove duplicates
Groundwater_result = pd.DataFrame(Groundwater_results).drop_duplicates()

# Reset index to remove the large index numbers
Groundwater_result = Groundwater_result.reset_index(drop=True)

# Aggregate by CLUSTER, SEASON, YEAR
result_groundwater_demand = (
    Groundwater_result
    .groupby(['CLUSTER', 'SEASON', 'YEAR'], as_index=False)['VALUE']
    .sum()
)

result_groundwater_demand = result_groundwater_demand[['CLUSTER', 'SEASON', 'VALUE', 'YEAR']]

# Sort 
result_groundwater_demand = result_groundwater_demand.sort_values(['CLUSTER', 'YEAR', 'SEASON']).reset_index(drop=True)

# Display the summary
display(result_groundwater_demand)

In [ ]:
# Final water supply

# Exit if the indicator is not of interest
if ws_aa == False:
    exit()

# Ensure all merge columns are of the same type (string for CLUSTER, int for YEAR, int for SEASON)
def _normalize_types(df):
    df = df.copy()
    df['CLUSTER'] = df['CLUSTER'].astype(str)
    df['YEAR'] = df['YEAR'].astype(int)
    # Convert SEASON to int if possible, else to string
    if df['SEASON'].dtype != 'int64' and df['SEASON'].dtype != 'Int64':
        df['SEASON'] = pd.to_numeric(df['SEASON'], errors='coerce').astype('Int64')
    return df

dfs = [result_groundwater_demand, result_Surface_water_demand]
dfs = [_normalize_types(df) for df in dfs]

# Rename VALUE columns to distinguish them
dfs[0] = dfs[0].rename(columns={'VALUE': 'VALUE_GROUNDWATER'})
dfs[1] = dfs[1].rename(columns={'VALUE': 'VALUE_SURFACEWATER'})

# Merge all  DataFrames on CLUSTER, SEASON, YEAR (outer join)
merged = pd.merge(dfs[0], dfs[1], on=['CLUSTER', 'SEASON', 'YEAR'], how='outer')

# Fill NaN with 0 for summing
for col in ['VALUE_GROUNDWATER', 'VALUE_SURFACEWATER']:
    if col in merged.columns:
        merged[col] = merged[col].fillna(0)

# Calculate total VALUE
merged['VALUE'] = merged['VALUE_GROUNDWATER'] + merged['VALUE_SURFACEWATER']

# Select and reorder columns
final_water_supply = merged[['CLUSTER', 'SEASON', 'YEAR', 'VALUE']].sort_values(['CLUSTER', 'YEAR', 'SEASON']).reset_index(drop=True)

display(final_water_supply)

In [ ]:


# Exit if the indicator is not of interest
if ws_aa == False:
    exit()

# Ensure numpy and pandas are available
import numpy as np
import pandas as pd

# Get unique seasons and clusters
seasons = sorted(final_water_demand['SEASON'].unique())
clusters = sorted(final_water_demand['CLUSTER'].unique())

for season in seasons:
    # Create season-specific output filenames
    season_suffix = f"_S{season}"
    ws_score_file = Annual_average_ws_score.replace('.csv', f'{season_suffix}.csv')
    r_file = r_csv.replace('.csv', f'{season_suffix}.csv')
    
    # Filter data for this season
    demand_season = final_water_demand[final_water_demand['SEASON'] == season]
    supply_season = final_water_supply[final_water_supply['SEASON'] == season]
    
    # Initialize lists to store results
    all_ws_results = []
    all_r_results = []
    
    for cluster in clusters:
        # If the cluster has no initial-stress entry, skip or handle as needed
        if cluster not in Initial_water_stress:
            print(f"Warning: initial water stress not found for cluster '{cluster}'. Skipping cluster.")
            continue
        init_ws_cluster = Initial_water_stress[cluster]
        
        # Filter data for this cluster
        demand_cluster = demand_season[demand_season['CLUSTER'] == cluster]
        supply_cluster = supply_season[supply_season['CLUSTER'] == cluster]
        
        # Merge the demand and supply data on YEAR 
        merged_df = pd.merge(
            demand_cluster,
            supply_cluster,
            on=["YEAR", "CLUSTER"],
            how="inner",
            suffixes=("_demand", "_supply")
        )
        
        if merged_df.empty:
            continue



        # Calculate relative water demand (r) and sort by YEAR
        merged_df["r"] = merged_df["VALUE_demand"] / merged_df["VALUE_supply"]
        merged_df = merged_df.sort_values("YEAR").reset_index(drop=True)
        
        # Safety: ensure the first r is non-zero to compute scaling factor
        first_r = merged_df.at[0, "r"]
        if first_r == 0 or np.isclose(first_r, 0.0):
            print(f"Warning: first r is zero for cluster '{cluster}' season {season} (year {merged_df.at[0,'YEAR']}). Skipping scaling for this cluster.")
            # Option A: skip cluster (as above)
            continue
            # Option B (alternative): set scaling_factor = 1.0
            # scaling_factor = 1.0
        
        # Calculate scaling factor using cluster-specific initial water stress
        scaling_factor = init_ws_cluster / first_r
        
        # Scale all r values
        merged_df["r"] = merged_df["r"] * scaling_factor
        
        # Calculate ws_aa using the formula (clipped between 0 and 1)
        merged_df["ws_aa"] = np.minimum(1, np.maximum(0, merged_df["r"]))
        
        # Calculate Average_Annual_score using the formula
        # Note: log of zero is invalid; we ensure ws_aa>0 where used
        # To avoid log(0), replace zeros with a tiny positive value for the score computation
        safe_ws = merged_df["ws_aa"].replace(0, 1e-12)
        merged_df["Average_Annual_score"] = np.maximum(
            0,
            np.minimum(
                5,
                ((np.log(safe_ws) - np.log(0.1)) / np.log(2)) + 1
            )
        )
        
        # Prepare results for this cluster
        # Water stress score results
        ws_cluster = merged_df[["YEAR", "Average_Annual_score"]].copy()
        ws_cluster["INDICATOR"] = "AverageAnnualWaterStressScore"
        ws_cluster["CLUSTER"] = cluster
        ws_cluster.rename(columns={"Average_Annual_score": "VALUE"}, inplace=True)
        ws_cluster = ws_cluster[["INDICATOR", "CLUSTER", "VALUE", "YEAR"]]
        ws_cluster["VALUE"] = ws_cluster["VALUE"].round(3)
        all_ws_results.append(ws_cluster)
        
        # Relative water demand results
        r_cluster = merged_df[["YEAR", "r"]].copy()
        r_cluster["INDICATOR"] = "r"
        r_cluster["CLUSTER"] = cluster
        r_cluster.rename(columns={"r": "VALUE"}, inplace=True)
        r_cluster = r_cluster[["INDICATOR", "CLUSTER", "VALUE", "YEAR"]]
        r_cluster["VALUE"] = r_cluster["VALUE"].round(3)
        all_r_results.append(r_cluster)
    
    # Combine results from all clusters for this season and save
    if all_ws_results:
        final_ws = pd.concat(all_ws_results, ignore_index=True)
        final_ws.to_csv(ws_score_file, index=False)
        print(f"Season {season} water stress scores saved to {ws_score_file}")
    else:
        print(f"No water stress score results for season {season}.")
    
    if all_r_results:
        final_r = pd.concat(all_r_results, ignore_index=True)
        final_r.to_csv(r_file, index=False)
        print(f"Season {season} relative water demand saved to {r_file}")
    else:
        print(f"No relative-demand results for season {season}.")



In [ ]:
tmp = final_water_supply.copy()
tmp["SEASON_NUM"] = tmp["SEASON"].astype(str).str.replace("S", "", regex=False).astype(int)

# show all matching rows
mask = (tmp["CLUSTER"] == "C04") & (tmp["SEASON_NUM"] == 2)
display(tmp[mask])

# show per-year sums
display(tmp[mask].groupby("YEAR", as_index=False)["VALUE"].sum())

# get value for a single year (e.g. 2020)
val_2020 = tmp[mask & (tmp["YEAR"] == 2020)]["VALUE"].sum()
print("VALUE for C04, S2, 2020:", val_2020)
